# Data Cleaning

After scraping the data, I needed to clean it up so that it was usable for our model. I made the following changes and created the following variables: Parsed numeric data out of salary,Made columns for employer provided salary and hourly wages,Removed rows without salary,Parsed rating out of company text,Made a new column for company state,Added a column for if the job was at the company’s headquarters,Transformed founded date into age of company
Made columns for if different skills were listed in the job description:
Python
R
Excel
AWS
Spark
Column for simplified job title and Seniority
Column for description length


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import os
os.chdir('/home/user')

In [5]:
df=pd.read_csv('glassdoor_jobs.csv')
# df.shape-This dataset contains record of 956 job openings

In [6]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [7]:
# What all do we need to do to clean the data?
# Salary parsing
# Company name text only
# State field
# Age of Company
# Parsing of Job Description(python etc.)



In [8]:
df.drop('Unnamed: 0',axis=1,inplace=True)

Salary Parsing

In [21]:
df.shape

(956, 14)

In [9]:
df=df[df['Salary Estimate']!='-1'] # Few salaries contain -1 so those values are not of much importance to us so let's remove them.
# As the salary estimate column is in string right now so we have to give it in string format

In [10]:
df.shape # So 956 rows get reduced down to 742 rows

(742, 14)

In [11]:
df.dtypes

Job Title             object
Salary Estimate       object
Job Description       object
Rating               float64
Company Name          object
Location              object
Headquarters          object
Size                  object
Founded                int64
Type of ownership     object
Industry              object
Sector                object
Revenue               object
Competitors           object
dtype: object

In [12]:
# So the next thing is removing the (Glassdoor est.) from the salary estiate column, so we can do that by 2 methods
# We can either do it using regular expressions and remove that part or else split on the left parantheses so that the left part will be remaining and the right part will be removed
# Or else we can replace those particular strings with empty values

In [13]:
salary = df['Salary Estimate'].apply(lambda x:x.split('(')[0])  # Removing (Glassdoor est.) from the values
salary.head()

0     $53K-$91K 
1    $63K-$112K 
2     $80K-$90K 
3     $56K-$97K 
4    $86K-$143K 
Name: Salary Estimate, dtype: object

In [15]:
minus_kd = salary.apply(lambda x:x.replace('K','').replace('$','')) # So here 53 implies $53K(53,000 dollars)-just to simplify it we ignore the K,$ symbols
minus_kd.head()

0     53-91 
1    63-112 
2     80-90 
3     56-97 
4    86-143 
Name: Salary Estimate, dtype: object

In [16]:
min_hr = minus_kd.apply(lambda x:x.lower().replace('per hour','').replace('employer provided salary:',''))          # Now we shouldn't have employer provided or per hour salaries

In [19]:
df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))                                                   # returns the list containing all the salaries present on the left hand side 
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))                                                   # returns the list containing all the salaries present on the right hand side 

In [20]:
df['min_salary'].dtype

dtype('int64')

In [21]:
df['avg_salary'] = (df.min_salary + df.max_salary)/2                                                                    # So now this becomes our dependent varible(we want to predict the salary of a particular person)
df['avg_salary'].head()

0     72.0
1     87.5
2     85.0
3     76.5
4    114.5
Name: avg_salary, dtype: float64

Company Name has to be displayed in text only

In [22]:
df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] <0 else x['Company Name'][:-3], axis = 1)
# We have few columns where the rating given is -1 so it doesn't have any meaning so that's why we take all those rows where the rating as -1 and then add the Company Name into a seperate column)

In [23]:
df['company_txt']=df['company_txt'].apply(lambda x:x.replace('\n','')) # Replace the \n with an empty string 

State Field

In [36]:
# Tells us the state where the job openings are present in

In [39]:
df['job_state']=df['Location'].apply(lambda x:x.split(',')[1]) # So we are splitting on the comma(,) so now this returns a list of all the states where the job openings are present
# Here if we give [0] then we get all the places in which job openings are present

In [41]:
df.job_state.value_counts()  # Here we get to know the number of jobs are present in each state

 CA             151
 MA             103
 NY              72
 VA              41
 IL              40
 MD              35
 PA              33
 TX              28
 WA              21
 NC              21
 NJ              17
 FL              16
 OH              14
 TN              13
 CO              11
 DC              11
 UT              10
 IN              10
 WI              10
 MO               9
 AZ               9
 AL               8
 DE               6
 KY               6
 MI               6
 GA               6
 CT               5
 IA               5
 LA               4
 NE               4
 OR               4
 NM               3
 KS               3
 ID               2
 MN               2
 Los Angeles      1
 SC               1
 RI               1
Name: job_state, dtype: int64

In [42]:
# Now we want to see if the job is present at the Headquarters or not

In [24]:
df['same_state']=df.apply(lambda x: 1 if x.Location==x.Headquarters else 0,axis=1) # This returns the list of states where the job openings are located in the headquarters,we want to compare the rows so that's why we put axis=1
# If the job openings present in the headquarters then we get a 1 or else a 0

Age of Company

In [25]:
# To find the age of the company we can subtract the year founded from the present year and if -1 is present then leave it

In [26]:
df['age']=df.Founded.apply(lambda x: 1 if x<0 else 2020-x)

Parsing of Job Descriptions-so we look for some of the relevant fields for our data analysis

In [27]:
df['Job Description'][0]

'Data Scientist\nLocation: Albuquerque, NM\nEducation Required: Bachelor’s degree required, preferably in math, engineering, business, or the sciences.\nSkills Required:\nBachelor’s Degree in relevant field, e.g., math, data analysis, database, computer science, Artificial Intelligence (AI); three years’ experience credit for Master’s degree; five years’ experience credit for a Ph.D\nApplicant should be proficient in the use of Power BI, Tableau, Python, MATLAB, Microsoft Word, PowerPoint, Excel, and working knowledge of MS Access, LMS, SAS, data visualization tools, and have a strong algorithmic aptitude\nExcellent verbal and written communication skills, and quantitative analytical skills are required\nApplicant must be able to work in a team environment\nU.S. citizenship and ability to obtain a DoD Secret Clearance required\nResponsibilities: The applicant will be responsible for formulating analytical solutions to complex data problems; creating data analytic models to improve data

In [28]:
# So we look for the main data science tools required which are Python,R,Apache Spark,Excel

In [29]:
df['python']=df['Job Description'].apply(lambda x:1 if 'python' in x.lower() else 0)
df['python'].value_counts() # So out of the 742 companies 392 comapanies require python and the other 350 have not included python which is actually shocking becuase python is the most preferred programming language for data science

1    392
0    350
Name: python, dtype: int64

In [30]:
df['r_studio']=df['Job Description'].apply(lambda x:1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
df['r_studio'].value_counts()

0    740
1      2
Name: r_studio, dtype: int64

In [31]:
df['spark']=df['Job Description'].apply(lambda x:1 if 'spark' in x.lower() else 0)
df['spark'].value_counts() # So many companies require Apache Spark as well

0    575
1    167
Name: spark, dtype: int64

In [32]:
df['aws']=df['Job Description'].apply(lambda x:1 if 'aws' in x.lower() else 0)
df['aws'].value_counts() # Similarly most of the companies focus on one of the most famous cloud computing platforms which is Amazon Web Services

0    566
1    176
Name: aws, dtype: int64

In [33]:
df['excel']=df['Job Description'].apply(lambda x:1 if 'excel' in x.lower() else 0)
df['excel'].value_counts() # Excel is not much required as data scientists do not use Excel these days

1    388
0    354
Name: excel, dtype: int64

In [34]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'min_salary', 'max_salary', 'avg_salary', 'company_txt', 'same_state',
       'age', 'python', 'r_studio', 'spark', 'aws', 'excel'],
      dtype='object')

In [39]:
df.to_csv('salary_data_cleaned.csv',index = False) # Exporting it to a csv file